In [ ]:
import geopandas as gpd
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
from aves.data import census
from pathlib import Path
from aves.features.geo import clip_area_geodataframe
from aves.visualization.figures import small_multiples_from_geodataframe
from aves.visualization.maps import choropleth_map

sns.set_context('paper')
sns.set_style('ticks')

https://mapas-conaset.opendata.arcgis.com/

https://mapas-conaset.opendata.arcgis.com/datasets/4b636b2f27164b0ebbeca0ab9db4d08a_0/explore

In [ ]:
gdf = gpd.read_file('https://opendata.arcgis.com/api/v3/datasets/4b636b2f27164b0ebbeca0ab9db4d08a_0/downloads/data?format=geojson&spatialRefId=4326&where=1%3D1', dtypes={'Fecha': 'str', 'Hora': 'str'})

In [ ]:
gdf

In [ ]:
gdf.plot()

In [ ]:
gdf.columns

In [ ]:
gdf[['COMUNAREAL', 'Comuna_1']].value_counts()

In [ ]:
gdf['Cód_Calza']

In [ ]:
gdf.columns

In [ ]:
gdf['Fallecidos'].value_counts()

In [ ]:
gdf['Graves'].value_counts()

In [ ]:
gdf['Menos_Grav'].value_counts()

In [ ]:
gdf['Ilesos'].value_counts()

In [ ]:
gdf['victimas'] = gdf[['Ilesos', 'Graves', 'Menos_Grav', 'Fallecidos']].sum(axis=1)

In [ ]:
gdf['Condición'].value_counts()

## ¿Cómo?

In [ ]:
gdf['Tipo_Accid'].value_counts()

In [ ]:
gdf['Tipo_Accid'].str.split().map(lambda x: x[0]).value_counts()

In [ ]:
gdf['tipo_simple'] = gdf['Tipo_Accid'].str.split().map(lambda x: x[0])
gdf['tipo_simple'].value_counts()

## ¿Cuándo?

In [ ]:
gdf['fecha'] = pd.to_datetime(gdf['Fecha'])

In [ ]:
gdf.resample('1w', on='fecha').size().plot()

In [ ]:

fig, axes = plt.subplots(1, 3, figsize=(14, 4))

mean_accidents = (
    lambda name, x: x.resample("1d", on="fecha")
    #.size()
    ['victimas'].sum()
    .rolling(14, center=True)
    .mean()
    .rename(name)
    .to_frame()
    .apply(lambda x: (x - x.mean()) / x.std())
)

relevant = [
    ["ATROPELLO", "CAIDA"],
    ["CHOQUE", "COLISION", "IMPACTO", "VOLCADURA"],
    ["INCENDIO", "OTRO"],
]

for ax, cats in zip(axes, relevant):
    # print(g)
    g = gdf[gdf["tipo_simple"].isin(cats)]
    mean_accidents("all", gdf).plot(ax=ax, color="grey", linewidth=0.5, legend=False)
    mean_accidents(cats[0], g).plot(ax=ax, linewidth=1.5, color='purple', legend=False)
    ax.set_title(cats[0])
    sns.despine(ax=ax, left=True, bottom=True)

fig.tight_layout()


## ¿Dónde?

In [ ]:
victimas_por_comuna = gdf.groupby(['Comuna_1', 'tipo_simple'])['victimas'].sum().unstack(fill_value=0)

In [ ]:
sns.clustermap(victimas_por_comuna, method='ward')

In [ ]:


AVES_ROOT = Path("../..")
comunas = census.read_census_map('comuna', path=AVES_ROOT / "data" / "external" / "censo_2017_R13").to_crs('epsg:5361')
zones = gpd.read_file(AVES_ROOT / "data" / "processed" / "scl_zonas_urbanas.json").set_index('ID').to_crs(comunas.crs)

comunas_urbanas = (
    comunas[comunas["COMUNA"].isin(zones["Com"].unique())]
    .drop("NOM_COMUNA", axis=1)
    .copy()
)

comunas_urbanas["NombreComuna"] = comunas_urbanas["COMUNA"].map(
    dict(zip(zones["Com"], zones["Comuna"].str.upper()))
)

bounding_box = zones.total_bounds
comunas_urbanas = clip_area_geodataframe(comunas_urbanas, zones.total_bounds, buffer=1000)

In [ ]:
comunas_urbanas['NombreComuna'] = comunas_urbanas['NombreComuna'].replace({'Á': 'A', 'Ú': 'U', 'Ó': 'O', 'Í': 'I', 'É': 'E'}, regex=True)

In [ ]:


fig, axes = small_multiples_from_geodataframe(comunas_urbanas, 3, height=7)

for cats, ax in zip(relevant, axes):
    acc = victimas_por_comuna[cats].sum(axis=1).rename(cats[0])
    choropleth_map(ax, comunas_urbanas.join(acc, on='NombreComuna', how='inner'), column=cats[0], binning="fisher_jenks", k=4,
    cbar_args=dict(
        label="Víctimas de Accidente",
        height="22%",
        width="2%",
        orientation="vertical",
        location="center left",
        label_size="medium",
        bbox_to_anchor=(0.0, 0.0, 0.9, 1.0),
        bin_spacing='uniform'
    ))
    ax.set_title(cats[0])

fig.tight_layout()

In [ ]:
accidentes_por_comuna = gdf.groupby(['Comuna_1', 'tipo_simple']).size().unstack(fill_value=0)
accidentes_por_comuna

In [ ]:
sns.clustermap((victimas_por_comuna + 1) / (accidentes_por_comuna + 1))

In [ ]:
accidentabilidad = ((victimas_por_comuna + 1) / (accidentes_por_comuna + 1))

In [ ]:
fig, axes = small_multiples_from_geodataframe(comunas_urbanas, 3, height=7)

for cats, ax in zip(relevant, axes):
    acc = accidentabilidad[cats].sum(axis=1).rename(cats[0])
    choropleth_map(ax, comunas_urbanas.join(acc, on='NombreComuna', how='inner'), column=cats[0], binning="fisher_jenks", k=4,
    cbar_args=dict(
        label="Accidentabilidad",
        height="22%",
        width="2%",
        orientation="vertical",
        location="center left",
        label_size="medium",
        bbox_to_anchor=(0.0, 0.0, 0.9, 1.0),
        bin_spacing='uniform'
    ))
    ax.set_title(cats[0])

fig.tight_layout()

## ¿Cómo proseguirían? La siguiente pregunta es ¿por qué?